In [10]:
import numpy as np
import pandas as pd
import time

df = pd.read_csv('/Users/noahv/Data-Mining-techniques/course_dmt/ass2/datasets/training_set_VU_DM.csv')

# Cleaning Training data

In [11]:
def remove_column(df : pd.DataFrame, column_name):
    df = df.drop(column_name, axis=1)
    return df


def add_nan_column(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df[str(col) + '_is_nan'] = df[col].isna().astype(np.int8)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("add_nan_column runtime: %.2f seconds" % elapsed_time)

    return df


def add_zero_column(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df[str(col) + '_is_zero'] = df[col].eq(0).astype(np.int8)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("add_zero_column runtime: %.2f seconds" % elapsed_time)

    return df


# Calculates the price per night where we know the length of stay is equal to 1 for each prop_id
def price_per_night_single_stays(df: pd.DataFrame) -> pd.DataFrame:
    mask = df['srch_length_of_stay'] == 1
    df['mean_price_per_night'] = df.groupby('prop_id')['price_usd'].transform(lambda x: x[mask].mean())
    return df


# def price_per_night_multiple_stays


# Function to impute the prop_location_score2 values that are NaN based on the prop_location_score2 values of the same prop_id
def impute_prop_loc_score2_using_future_score2(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Retrieve the prop_ids that have a change in prop_location_score2 from NaN to a float over time
    has_change_over_time = df.groupby('prop_id')['prop_location_score2'].apply(lambda x: x.diff().notna().any())
    prop_ids_change = has_change_over_time[has_change_over_time == True].index

    # For each prop_id that has a change in prop_location_score2 from NaN to a float over time, impute the NaN values with the first non-NaN value
    for prop_id in prop_ids_change:
        mask = (df['prop_id'] == prop_id) & (df['prop_location_score2'].isnull())
        non_nan_value = df.loc[(df['prop_id'] == prop_id) & (df['prop_location_score2'].notna()), 'prop_location_score2'].values[0]
        df.loc[mask, 'prop_location_score2'] = non_nan_value

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_loc_score2_using_future_score2 runtime: %.2f seconds" % elapsed_time)

    return df


# Impute remaning NaN values using the mean prop_locatioin_score2 values for each prop_location_score1 value
def impute_prop_location_score2_using_mean_score1(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Create an extra column with rounded 'prop_location_score1'
    df['rounded_score1'] = df['prop_location_score1'].round()

    # Calculate the mean of 'prop_location_score2' for each rounded integer value
    mean_score2_by_rounded = df.groupby('rounded_score1')['prop_location_score2'].mean()

    # Impute missing values of 'prop_location_score2' based on rounded integer values
    df['prop_location_score2'].fillna(df['rounded_score1'].map(mean_score2_by_rounded), inplace=True)

    # Drop the extra column 'rounded_score1' if no longer needed
    df.drop('rounded_score1', axis=1, inplace=True)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_location_score2_using_mean_score1 runtime: %.2f seconds" % elapsed_time)

    return df


def impute_prop_review_score(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Calculate the mean of 'prop_review_score' for each rounded integer value
    mean_review_score_for_starrating = df.groupby('prop_starrating')['prop_review_score'].mean()

    # Impute missing values of 'prop_review_score' based on rounded integer values
    df['prop_review_score'].fillna(df['prop_starrating'].map(mean_review_score_for_starrating), inplace=True)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_review_score runtime: %.2f seconds" % elapsed_time)

    return df


#def impute_prop_starrating(df: pd.DataFrame):

#def impute_query_affinity_score(df: pd.DataFrame):


# Calculate the mean distance per hotel and impute the NaN values with the mean distance
def impute_orig_destination_distance(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Calculate the mean distance per hotel
    mean_distance_per_hotel = df.groupby('prop_id')['orig_destination_distance'].mean()
    
    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['prop_id'].map(mean_distance_per_hotel), inplace=True)

    # BUT there are also hotels that contain only NaN values for orig_destination_distance, 
    # for these we impute it with the mean distance for the country_id
    # Use visitor_location_country_id or country_id?

    # Calculate the mean distance per country
    mean_distance_per_country = df.groupby('visitor_location_country_id')['orig_destination_distance'].mean()

    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['visitor_location_country_id'].map(mean_distance_per_country), inplace=True)
    
    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_orig_destination_distance runtime: %.2f seconds" % elapsed_time)

    return df


def impute_and_transform_query_affinity_score(df : pd.DataFrame) -> pd.DataFrame:

    df['srch_query_affinity_score'] = np.power(2, df['srch_query_affinity_score'])
    df['srch_query_affinity_score'].fillna(0, inplace=True)

    return df


def define_target(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    df['target'] = df['click_bool'] +  df['booking_bool'] * 5

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("define_target runtime: %.2f seconds" % elapsed_time)

    return df


def one_hot_encode(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df = pd.get_dummies(df, columns=[col], prefix=col)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("one_hot_encode runtime: %.2f seconds" % elapsed_time)

    return df


def delete_id_columns(df: pd.DataFrame)-> pd.DataFrame:
    start_time = time.time()

    for col in df.columns:
        if col.endswith('_id'):
            df = df.drop(col, axis=1)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("delete_id_columns runtime: %.2f seconds" % elapsed_time)

    return df


def drop_nan_columns(df: pd.DataFrame)-> pd.DataFrame:
    start_time = time.time()

    for col in df.columns:
        if df[col].isna().sum() > 0:
            #print(col)
            df = df.drop(col, axis=1)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("drop_nan_columns runtime: %.2f seconds" % elapsed_time)

    return df


def print_columns_containing_string(df : pd.DataFrame):
    matching_columns = [col for col in df.columns if df[col].dtype == 'object']
    if matching_columns:
        print("Columns containing string values:")
        for col in matching_columns:
            print(col)


def remove_column(df : pd.DataFrame, column_name):
    df = df.drop(column_name, axis=1)
    return df


#rescale all columns to [0,1]
def rescaler(df : pd.DataFrame):
    for col in df.columns:
        # except for the id columns
        if not col.endswith('_id'):
            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df


def main(df : pd.DataFrame) -> pd.DataFrame:
    #df = remove_column(df, 'date_time')
    df = add_nan_column(df, ['prop_review_score', 'srch_query_affinity_score'])
    df = add_zero_column(df, ['prop_review_score', 'prop_starrating'])
    #df = price_per_night_single_stays(df)
    # df = price_per_night_multiple_stays
    # df = impute_prop_loc_score2_using_future_score2(df)
    df = impute_prop_location_score2_using_mean_score1(df)
    df = impute_prop_review_score(df)
    df = impute_orig_destination_distance(df)
    df = impute_and_transform_query_affinity_score(df)
    df = define_target(df)

    # With XGBoost no need to one-hot encode en delete id columns
    # df = one_hot_encode(df, ['prop_country_id', 'visitor_location_country_id','prop_id'])
    # df = delete_id_columns(df)

    df = drop_nan_columns(df)
    #df = print_columns_containing_string(df)
    #df = rescaler(df)
    df.to_csv('data_cleaned.csv', index=False)
    return df

df = pd.read_csv('/Users/noahv/Data-Mining-techniques/course_dmt/ass2/datasets/training_set_VU_DM.csv')
df = main(df)
df

add_nan_column runtime: 0.32 seconds
add_zero_column runtime: 0.14 seconds
impute_prop_location_score2_using_mean_score1 runtime: 9.23 seconds
impute_prop_review_score runtime: 0.74 seconds
impute_orig_destination_distance runtime: 1.84 seconds
define_target runtime: 0.11 seconds
drop_nan_columns runtime: 46.55 seconds


,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,booking_bool,prop_review_score_is_nan,srch_query_affinity_score_is_nan,prop_review_score_is_zero,prop_starrating_is_zero,target
0,1,2013-04-04 08:32:15,12,187,219,893,3,3.5,1,2.83,...,0.0,1776.833608,1,0,0,0,1,0,0,0
1,1,2013-04-04 08:32:15,12,187,219,10404,4,4.0,1,2.20,...,0.0,1760.058186,1,0,0,0,1,0,0,0
2,1,2013-04-04 08:32:15,12,187,219,21315,3,4.5,1,2.20,...,0.0,1760.820221,1,0,0,0,1,0,0,0
3,1,2013-04-04 08:32:15,12,187,219,27348,2,4.0,1,2.83,...,0.0,1786.456451,1,0,0,0,1,0,0,0
4,1,2013-04-04 08:32:15,12,187,219,29604,4,3.5,1,2.64,...,0.0,1857.881111,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,2013-06-30 19:55:18,5,219,219,77700,3,4.0,1,1.61,...,0.0,550.920000,0,0,0,0,1,0,0,0
4958343,332785,2013-06-30 19:55:18,5,219,219,88083,3,4.0,1,1.95,...,0.0,553.140000,0,0,0,0,1,0,0,0
4958344,332785,2013-06-30 19:55:18,5,219,219,94508,3,3.5,1,1.10,...,0.0,544.430000,0,0,0,0,1,0,0,0
4958345,332785,2013-06-30 19:55:18,5,219,219,128360,3,5.0,1,1.95,...,0.0,550.380000,0,1,1,0,1,0,0,6


# Cleaning test data

In [13]:
def remove_column(df : pd.DataFrame, column_name):
    df = df.drop(column_name, axis=1)
    return df


def add_nan_column(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df[str(col) + '_is_nan'] = df[col].isna().astype(np.int8)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("add_nan_column runtime: %.2f seconds" % elapsed_time)

    return df


def add_zero_column(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df[str(col) + '_is_zero'] = df[col].eq(0).astype(np.int8)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("add_zero_column runtime: %.2f seconds" % elapsed_time)

    return df


# Calculates the price per night where we know the length of stay is equal to 1 for each prop_id
def price_per_night_single_stays(df: pd.DataFrame) -> pd.DataFrame:
    mask = df['srch_length_of_stay'] == 1
    df['mean_price_per_night'] = df.groupby('prop_id')['price_usd'].transform(lambda x: x[mask].mean())
    return df


# def price_per_night_multiple_stays


# Function to impute the prop_location_score2 values that are NaN based on the prop_location_score2 values of the same prop_id
def impute_prop_loc_score2_using_future_score2(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Retrieve the prop_ids that have a change in prop_location_score2 from NaN to a float over time
    has_change_over_time = df.groupby('prop_id')['prop_location_score2'].apply(lambda x: x.diff().notna().any())
    prop_ids_change = has_change_over_time[has_change_over_time == True].index

    # For each prop_id that has a change in prop_location_score2 from NaN to a float over time, impute the NaN values with the first non-NaN value
    for prop_id in prop_ids_change:
        mask = (df['prop_id'] == prop_id) & (df['prop_location_score2'].isnull())
        non_nan_value = df.loc[(df['prop_id'] == prop_id) & (df['prop_location_score2'].notna()), 'prop_location_score2'].values[0]
        df.loc[mask, 'prop_location_score2'] = non_nan_value

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_loc_score2_using_future_score2 runtime: %.2f seconds" % elapsed_time)

    return df


# Impute remaning NaN values using the mean prop_locatioin_score2 values for each prop_location_score1 value
def impute_prop_location_score2_using_mean_score1(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Create an extra column with rounded 'prop_location_score1'
    df['rounded_score1'] = df['prop_location_score1'].round()

    # Calculate the mean of 'prop_location_score2' for each rounded integer value
    mean_score2_by_rounded = df.groupby('rounded_score1')['prop_location_score2'].mean()

    # Impute missing values of 'prop_location_score2' based on rounded integer values
    df['prop_location_score2'].fillna(df['rounded_score1'].map(mean_score2_by_rounded), inplace=True)

    # Drop the extra column 'rounded_score1' if no longer needed
    df.drop('rounded_score1', axis=1, inplace=True)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_location_score2_using_mean_score1 runtime: %.2f seconds" % elapsed_time)

    return df


def impute_prop_review_score(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Calculate the mean of 'prop_review_score' for each rounded integer value
    mean_review_score_for_starrating = df.groupby('prop_starrating')['prop_review_score'].mean()

    # Impute missing values of 'prop_review_score' based on rounded integer values
    df['prop_review_score'].fillna(df['prop_starrating'].map(mean_review_score_for_starrating), inplace=True)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_prop_review_score runtime: %.2f seconds" % elapsed_time)

    return df


#def impute_prop_starrating(df: pd.DataFrame):

#def impute_query_affinity_score(df: pd.DataFrame):


# Calculate the mean distance per hotel and impute the NaN values with the mean distance
def impute_orig_destination_distance(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    # Calculate the mean distance per hotel
    mean_distance_per_hotel = df.groupby('prop_id')['orig_destination_distance'].mean()
    
    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['prop_id'].map(mean_distance_per_hotel), inplace=True)

    # BUT there are also hotels that contain only NaN values for orig_destination_distance, 
    # for these we impute it with the mean distance for the country_id
    # Use visitor_location_country_id or country_id?

    # Calculate the mean distance per country
    mean_distance_per_country = df.groupby('visitor_location_country_id')['orig_destination_distance'].mean()

    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['visitor_location_country_id'].map(mean_distance_per_country), inplace=True)
    
    # Print the runtime
    elapsed_time = time.time() - start_time
    print("impute_orig_destination_distance runtime: %.2f seconds" % elapsed_time)

    return df


def impute_and_transform_query_affinity_score(df : pd.DataFrame) -> pd.DataFrame:

    df['srch_query_affinity_score'] = np.power(2, df['srch_query_affinity_score'])
    df['srch_query_affinity_score'].fillna(0, inplace=True)

    return df


def define_target(df : pd.DataFrame) -> pd.DataFrame:
    start_time = time.time()

    df['target'] = df['click_bool'] +  df['booking_bool'] * 5

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("define_target runtime: %.2f seconds" % elapsed_time)

    return df


def one_hot_encode(df : pd.DataFrame, cols):
    start_time = time.time()

    for col in cols:
        df = pd.get_dummies(df, columns=[col], prefix=col)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("one_hot_encode runtime: %.2f seconds" % elapsed_time)

    return df


def delete_id_columns(df: pd.DataFrame)-> pd.DataFrame:
    start_time = time.time()

    for col in df.columns:
        if col.endswith('_id'):
            df = df.drop(col, axis=1)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("delete_id_columns runtime: %.2f seconds" % elapsed_time)

    return df


def drop_nan_columns(df: pd.DataFrame)-> pd.DataFrame:
    start_time = time.time()

    for col in df.columns:
        if df[col].isna().sum() > 0:
            #print(col)
            df = df.drop(col, axis=1)

    # Print the runtime
    elapsed_time = time.time() - start_time
    print("drop_nan_columns runtime: %.2f seconds" % elapsed_time)

    return df


def print_columns_containing_string(df : pd.DataFrame):
    matching_columns = [col for col in df.columns if df[col].dtype == 'object']
    if matching_columns:
        print("Columns containing string values:")
        for col in matching_columns:
            print(col)


def remove_column(df : pd.DataFrame, column_name):
    df = df.drop(column_name, axis=1)
    return df


#rescale all columns to [0,1]
def rescaler(df : pd.DataFrame):
    for col in df.columns:
        # except for the id columns
        if not col.endswith('_id'):
            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df


def main(df : pd.DataFrame) -> pd.DataFrame:
    #df = remove_column(df, 'date_time')
    df = add_nan_column(df, ['prop_review_score', 'srch_query_affinity_score'])
    df = add_zero_column(df, ['prop_review_score', 'prop_starrating'])
    #df = price_per_night_single_stays(df)
    # df = price_per_night_multiple_stays
    # df = impute_prop_loc_score2_using_future_score2(df)
    df = impute_prop_location_score2_using_mean_score1(df)
    df = impute_prop_review_score(df)
    df = impute_orig_destination_distance(df)
    df = impute_and_transform_query_affinity_score(df)

    # Cannot add target to the test set
    #df = define_target(df)

    # With XGBoost no need to one-hot encode en delete id columns
    # df = one_hot_encode(df, ['prop_country_id', 'visitor_location_country_id','prop_id'])
    # df = delete_id_columns(df)

    df = drop_nan_columns(df)
    #df = print_columns_containing_string(df)
    #df = rescaler(df)
    df.to_csv('test_data_cleaned.csv', index=False)
    return df

df = pd.read_csv('/Users/noahv/Data-Mining-techniques/course_dmt/ass2/datasets/test_set_VU_DM.csv')
df = main(df)
df

add_nan_column runtime: 0.42 seconds
add_zero_column runtime: 0.21 seconds
impute_prop_location_score2_using_mean_score1 runtime: 7.67 seconds
impute_prop_review_score runtime: 0.67 seconds
impute_orig_destination_distance runtime: 1.62 seconds
drop_nan_columns runtime: 34.68 seconds


,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,random_bool,prop_review_score_is_nan,srch_query_affinity_score_is_nan,prop_review_score_is_zero,prop_starrating_is_zero
0,1,2013-02-02 15:27:40,24,216,219,3180,3,4.5,1,2.94,...,2,0,1,0,0.0,0,0,1,0,0
1,1,2013-02-02 15:27:40,24,216,219,5543,3,4.5,1,2.64,...,2,0,1,0,0.0,0,0,1,0,0
2,1,2013-02-02 15:27:40,24,216,219,14142,2,3.5,1,2.71,...,2,0,1,0,0.0,0,0,1,0,0
3,1,2013-02-02 15:27:40,24,216,219,22393,3,4.5,1,2.40,...,2,0,1,0,0.0,0,0,1,0,0
4,1,2013-02-02 15:27:40,24,216,219,24194,3,4.5,1,2.94,...,2,0,1,0,0.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,2013-05-21 11:06:37,24,216,117,32019,4,3.5,0,2.48,...,1,0,1,0,0.0,0,0,1,0,0
4959179,332787,2013-05-21 11:06:37,24,216,117,33959,4,3.0,1,2.20,...,1,0,1,0,0.0,0,0,1,0,0
4959180,332787,2013-05-21 11:06:37,24,216,117,35240,4,0.0,0,1.79,...,1,0,1,0,0.0,0,0,1,1,0
4959181,332787,2013-05-21 11:06:37,24,216,117,94437,4,0.0,0,2.94,...,1,0,1,0,0.0,0,0,1,1,0
